# Inversion of a linear regressor

In [ ]:
import numpy as np
from regressor import LinearRegressor

import inversion_ideas as ii

## Synthetic data

Build synthetic data for a linear regressor such as:

$$ \mathbf{d}_\text{obs} = \mathbf{X} \mathbf{m}_\text{true} + \epsilon $$

where 
$\mathbf{d}_\text{obs}$ is the observed data (synthetic),
$\mathbf{X}$ is a matrix we randomly generate,
and $\mathbf{m}_\text{true}$ is the true model we'll try to invert for,
and $\epsilon$ is some random noise generated out of a normal distribution.

In [ ]:
n_params = 10
rng = np.random.default_rng(seed=4242)
true_model = rng.uniform(size=10)
true_model

In [ ]:
# Build the X matrix
n_data = 25
shape = (n_data, n_params)
X = rng.uniform(size=n_data * n_params).reshape(shape)

In [ ]:
# Generate synthetic data with noise
synthetic_data = X @ true_model
maxabs = np.max(np.abs(synthetic_data))
std_err =  1e-2 * maxabs
noise = rng.normal(scale=std_err, size=synthetic_data.size)
synthetic_data += noise
synthetic_data

## Inversion

In [ ]:
simulation = LinearRegressor(X)

In [ ]:
uncertainty = std_err * np.ones_like(synthetic_data)
data_misfit = ii.DataMisfit(synthetic_data, uncertainty, simulation)
data_misfit

In [ ]:
smallness = ii.TikhonovZero(n_params)
smallness

In [ ]:
phi = data_misfit + 1e-3 * smallness
phi

In [ ]:
initial_model = np.zeros(n_params)
initial_model

In [ ]:
from scipy.optimize import minimize

result = minimize(phi, initial_model)
result

In [ ]:
inverted_model = result.x
inverted_model

In [ ]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)

### Use the `Minimizer` class

In [ ]:
minimizer = ii.ConjugateGradient()
minimizer

In [ ]:
inverted_model = minimizer(phi, initial_model)
inverted_model

In [ ]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)